# Execute the process and monitor the execution

To submit an execution request of a deployed process and monitor it, the OGC API Processes API uses the resource highlighted in the table below:

| **Resource**                   | **Path**                                  | **Purpose**                                                                     | **Part**   |
|--------------------------------|-------------------------------------------|---------------------------------------------------------------------------------|------------|
| Landing page                   | `/`                                       | Top-level resource serving as an entry point.                                   | Part 1     |
| Conformance declaration        | `/conformance`                            | Information about the functionality supported by the server.                    | Part 1     |
| API Definition                 | `/api`                                    | Metadata about the API itself.                                                  | Part 1     |
| Process list                   | `/processes`                              | Lists available processes with identifiers and links to descriptions.           | Part 1     |
| Process description            | `/processes/{processID}`                  | Retrieves detailed information about a specific process.                        | Part 1     |
| Process execution              | `/processes/{processID}/execution`        | Executes a process, creating a job.                                             | Part 1     |
| Deploy Process                 | `/processes` (POST)                       | Deploys a new process on the server.                                            | Part 2     |
| Replace Process                | `/processes/{processID}` (PUT)            | Replaces an existing process with a new version.                                | Part 2     |
| Undeploy Process               | `/processes/{processID}` (DELETE)         | Removes an existing process from the server.                                    | Part 2     |
| EO Application Package         | `/processes/{processID}/package` (GET)    | Get the EOAP associated with a deployed process.                                | Part 2     |
| Job status info                | `/jobs/{jobID}`                           | Retrieves the current status of a job.                                          | Part 1     |
| Job results                    | `/jobs/{jobID}/results`                   | Retrieves the results of a job.                                                 | Part 1     |
| **Job list**                   | **`/jobs`**                               | **Retrieves a list of submitted jobs.**                                         | **Part 1** |
| **Job deletion**               | **`/jobs/{jobID}`**                       | **Cancels and deletes a job.**                                                  | **Part 1** |

In [ ]:
import json
from ogc_api_client.api_client import Configuration, ApiClient
from ogc_api_client.api_client_wrapper import ApiClientWrapper
from typing import Dict, Optional
from ogc_api_client.models.status_info import StatusInfo, StatusCode
from ogc_api_client.api.status_api import StatusApi
from ogc_api_client.api.job_list_api import JobListApi
from ogc_api_client.api.dismiss_api import DismissApi


namespace = "acme"

# ogc_api_endpoint = f"http://zoo-project-dru-service/{namespace}/ogc-api"
ogc_api_endpoint = f"http://localhost:8080/{namespace}/ogc-api"

configuration = Configuration(
    host=ogc_api_endpoint,
)
client = ApiClient(configuration=configuration)

Submit a job as in notebook 04:

In [ ]:
data = {
    "inputs": {
        "stac_items": [
            "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A",
            "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A"
        ],
        "aoi": "-121.399,39.834,-120.74,40.472",
        "epsg": "EPSG:4326",
        "bands": [
            "green",
            "nir"
        ]
    }
}

headers = {
    "accept": "*/*",
    "Prefer": "respond-async;return=representation",
    "Content-Type": "application/json"
}

process_id = "water-bodies"  # Replace with your process ID if different

# Submit the processing request
response_data = client.rest_client.request(
    "POST", f"{ogc_api_endpoint}/processes/{process_id}/execution",
    headers=headers,
    body=data,
)

_response_types_map: Dict[str, Optional[str]] = {
    '200': "Execute200Response",
    '201': "StatusInfo",
    '404': "Exception",
    '500': "Exception",
}

response_data.read()
content = client.response_deserialize(
    response_data=response_data,
    response_types_map=_response_types_map,
).data

if isinstance(content, StatusInfo):
    # Parse the response to get the job ID
    job_id = content.job_id
    print(f"Job submitted successfully. Job ID: {job_id}")
    status_location = next((link.href for link in content.links if link.rel == 'monitor'), None)
    if not status_location:
        status_location = f"{ogc_api_endpoint}/jobs/{job_id}"
                           
    print(f"Monitor job status at: {status_location}")
else:
    print(f"Failed to submit job. Status code: {response_data.status}")
    print("Response:", content.text)
    raise ValueError(f"Failed to submit job. Status code: {response_data.status}")


Request the list of jobs

In [ ]:
print("\nJob list...")

job_list_api = JobListApi(api_client=client)
status_api = StatusApi(api_client=client)

#job_list = job_list_api.get_jobs()

#for job in job_list.jobs:
#    print(f"Job: {job.job_id} ({job.process_id}), status: {job.status}")

print("\nJob status...")

try:
    status = status_api.get_status(job_id=job_id)
    print(status.status)
except Exception as e:
    print(f"ERROR: {str(e)}")


Delete (dismiss) running job

In [ ]:
dismiss_api = DismissApi(api_client=client)
dismiss_api.dismiss(job_id=job_id)

Check whether job still exists

In [ ]:
print("\nJob list...")

#job_list = job_list_api.get_jobs()

#for job in job_list.jobs:
#    print(f"Job: {job.job_id} ({job.process_id}), status: {job.status}")

print("\nJob status...")

try:
    status = status_api.get_status(job_id=job_id)
    print(status.status)
except Exception as e:
    print(f"ERROR: {str(e)}")
